In [1]:
import pandas as pd

In [11]:
import numpy as np

In [6]:
%matplotlib inline

In [2]:
df = pd.read_csv("BPD_Arrests.csv")

df2012 = pd.read_csv("BPD_Arrests_2012.csv")

In [55]:
df.head()

,Arrest,Age,Sex,Race,ArrestDate,ArrestTime,ArrestLocation,IncidentOffense,IncidentLocation,Charge,ChargeDescription,District,Post,Neighborhood,Longitude,Latitude,Location 1
0,17201066.0,30.0,M,B,12/23/2017,23:24,900 STOLL ST,4ECOMMON ASSAULT,900 STOLL ST,1 1415,2ND DEG ASSAULT,Southern,913.0,Brooklyn,-76.598589,39.230243,"(39.230243000000, -76.598589000000)"
1,17201085.0,19.0,M,B,12/23/2017,23:05,NaN,Unknown Offense,NaN,1 0521,THEFT LESS THAN $100.00,NaN,NaN,NaN,NaN,NaN,NaN
2,17201081.0,19.0,M,B,12/23/2017,23:05,NaN,Unknown Offense,NaN,3 4025,"MAL DEST PROP/VALU - $1,000",NaN,NaN,NaN,NaN,NaN,NaN
3,17201084.0,36.0,M,U,12/23/2017,23:00,300 MACON ST,Unknown Offense,300 MACON ST,3 4025,DESTRUCTION OF PROPERTY,Southeast,242.0,Greektown,-76.557454,39.288730,"(39.288730000000, -76.557454000000)"
4,17201068.0,47.0,M,B,12/23/2017,23:00,500 ARCHER ST,4ECOMMON ASSAULT,500 ARCHER ST,1 1415,2ND DEGREE ASSAULT,Southern,932.0,Washington Village,-76.630115,39.282715,"(39.282715000000, -76.630115000000)"


In [60]:
print(df2012['incidentOffense'].isnull().sum())
incident2012 = df2012['incidentOffense'].value_counts().to_frame()
incident2012['Percentage'] = incident2012.incidentOffense/incident2012.incidentOffense.sum()

0


In [58]:
print('null', df['IncidentOffense'].isnull().sum())
incident = df['IncidentOffense'].value_counts().to_frame()
incident['Percentage'] = incident.IncidentOffense/incident.IncidentOffense.sum()

null 0


In [61]:
incident.to_csv('tmp-arrest.csv')
incident2012.to_csv('tmp-arrest-2012.csv')

In [30]:
def hist_values(array, bins=None):
    if bins is None:
        count, division = np.histogram(array)
    else: 
        count, division = np.histogram(array, bins=bins)
    for i in range(len(count)):
        print('%d~%d\t%d' %(division[i], division[i+1], count[i]))

In [ ]:
hist_values(df[~df.Age.isnull()].Age, bins=[0, 14, 21, 28, 35,42,49,56,63,70,77,84,100])

In [ ]:
hist_values(df2012[~df2012.age.isnull()].age, bins=[0, 14, 21, 28, 35,42,49,56,63,70,77,84,100])